# **Introduction:**

This file serves to host an attempted Keras implementation of an Adaptive Neuro-Fuzzy Inference System (ANFIS).

**Date Created:** 22/01/2025

**Date Modified:** 27/01/2025

# **Import Packages:**

This section imports all the necessary packages for the ANFIS implementation.

In [537]:
# import packages:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import product
from keras.layers import Layer

# **Function & Layer Definition:**

This section creates the necessary custom functions and layers for this ANFIS implementation within Keras. 

In [1]:
# need to first define the initial layer -> the membership function layer:
class MembershipFunctionLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, params = None, **kwargs):   # by including **kwargs, we allow for additional arguments from keras, like name or dtype
        super(MembershipFunctionLayer, self).__init__(**kwargs)         # we are subclassing from the keras layer -> telling the constructor to make our layer like a keras layer
        self.num_inputs = num_inputs            # define the number of inputs to the ANFIS 
        self.num_mfs = num_mfs                  # define the number of membership functions per input
        self.num_rules = num_mfs ** num_inputs  # the number of rules is calculated as such:

        # next is the initialization of the antecedent parameters:
        if params is not None:
            # initialize custom parameters defined by the user:
            self.mf_params = self.add_weight(
                shape=(self.num_inputs, self.num_mfs, 3),       # define their shape, (num_inputs, num_mfs, 3) as we have 3 params for a triangular mf
                initializer=tf.constant_initializer(params),    # initialize as constants from the provided array
                trainable=True,                                 # set to trainable
                name="Antecedent Params",                       # assign them a name
            )
            print('Custom parameters have been set.')
        else:
            # initialize raw membership parameters:
            raw_params = self.add_weight(
                shape = (self.num_inputs, self.num_mfs, 3),     # define their shape, (num_inputs, num_mfs, 3) as we have 3 params for a triangular mf
                initializer = "random_uniform",                 # initialize as a random, uniform distribution
                trainable = True,                               # set to trainable
                name = "Raw Antecedent Params",                 # assign them a name 
            )

            # sort the parameters such that a <= b <= c:
            sorted_params = tf.sort(raw_params, axis = 1)           # sort such that a <= b <= c
            self.mf_params = self.add_weight(                       
                shape = (self.num_inputs, self.num_mfs, 3),                     # set the shape: (num_inputs, num_mfs, 3) as we have 3 params for a triangular mf
                initializer = tf.constant_initializer(sorted_params.numpy()),   # initialize as the sorted array of params
                trainable = True,                                               # set to trainable
                name = 'Antecedent Params'                                      # assign them a name
            )
            print('Random parameters have been set.')
        
        # need to also initialize the consequent parameters:
        self.consequent_params = self.add_weight(
            shape = (self.num_rules, self.num_inputs + 1),
            initializer = 'random_uniform',
            trainable = True,
            name = 'Consequent Params'
        )

    # define the triangular membership function within this layer as this is where it is used:
    def triangular_membership(self, x, params):
        a, b, c = params        # load params

        # throw error if not a < b < c:
        if a > b or b > c:
            raise ValueError("Invalid parameters: Ensure a < b < c.") 
    
        if a == b:  # rising ramp (plateau at b, c)
            return np.maximum(0, np.minimum(1, (c - x) / (c - b)))          
        elif b == c:  # falling ramp (plateau at a, b)
            return np.maximum(0, np.minimum(1, (x - a) / (b - a)))
        
        # general triangular shape:
        return np.maximum(0, np.minimum((x - a) / (b - a), (c - x) / (c - b)))
    
    def plot_mf(self, max_values, mf_names = None):
        # if the user did not provide names:
        if mf_names is None:
            mf_names = [f'MF {i + 1}' for i in range(self.num_mfs)]
        
        # make sure that the number of names matches the number of membership functions:
        if len(mf_names) != self.num_mfs:
            raise ValueError(f'Expected {self.num_mfs} membership functions, but got {len(mf_names)} instead.')
        
        # make sure that the provided max values match the number of membership functions:
        if len(max_values) != self.num_mfs:
            raise ValueError(f'Expected {self.num_mfs} max values, but got {len(max_values)} instead.') 
        
        # create linspace based on max values:
        input_range = {}
        for i in range(self.num_inputs):
            input_range[i] = np.linspace(0, max_values[i], 1000)

        # plot the mfs:
        for input_index in range(self.num_inputs):
            x_values = input_range[input_index]
            plt.figure(figsize = (12,8))

            # plot each mf for the selected input:
            for i in range(self.num_mfs):
                params = self.mf_params[input_index, i].numpy()
                y_values = [self.triangular_membership(x, params) for x in x_values]
                plt.plot(x_values, y_values, label = f'{mf_names[i]}')
                plt.title(f'Membership Functions for Input X{input_index + 1}')
                plt.xlabel('Input Value')
                plt.ylabel('Degree of Membership')
                plt.legend()
                plt.grid(True)
        
        plt.show()
    
    # need to define the call -> this is what gets executed by the layer:
    def call(self, inputs):
        # initialize list to hold membership values:
        membership_values = []

        # loop through each input:
        for i in range(self.num_inputs):
            input_values = inputs[:, i]   # for a given column, everything in the row
            mf_values = []                # initialize list for the MF values of this input

            # for every membership function:
            for j in range(self.num_mfs):
                params = self.mf_params[i, j].numpy()  # extract params

                mf_values.append(np.array([self.triangular_membership(x, params) for x in input_values]))

            membership_values.append(np.stack(mf_values, axis=-1))  # stack MFs for this input

        # combine memberships for all inputs into a single tensor:
        membership_values = tf.convert_to_tensor(np.stack(membership_values, axis = 1), dtype = tf.float32)

        return membership_values

# need to now define the second layer -> the firing strength layer:
class FiringStrengthLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, **kwargs):
        super(FiringStrengthLayer, self).__init__(**kwargs)
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs

    # call function:
    def call(self, membership_values):
        # get batch size:
        batch_size = tf.shape(membership_values)[0]  

        # initialize firing strengths
        firing_strengths = tf.ones((batch_size, self.num_rules), dtype = tf.float32)    

        # generate all rule combinations:
        rules = list(product(range(self.num_mfs), repeat = self.num_inputs))  # example: [(0, 0, 0), (0, 0, 1), ...]

        # need to check each input, each mf combination, and multiply their values together:
        for rule_index, combination in enumerate(rules):
            # print(f'rule: {rule_index + 1} | combination: {combination}')
            rule_strength = tf.ones((batch_size, ), dtype = tf.float32)

            for input_index, mf_index in enumerate(combination):
                # print(f'input: {input_index + 1} | mf: {mf_index + 1}')

                # correctly extract the fuzzified values based on the combination index:
                rule_strength *= membership_values[:, input_index, mf_index]
                # print(f'strength: {rule_strength}')
            
            # update the firing strengths:
            firing_strengths = tf.tensor_scatter_nd_update(
                firing_strengths,
                indices = [[i, rule_index] for i in range(batch_size)],
                updates = rule_strength
            )

        return firing_strengths

# need to now define the third layer -> the normalization layer:
class NormalizationLayer(Layer):
    # constructor:
    def __init__(self, num_inputs, num_mfs, **kwargs):
        super(NormalizationLayer, self).__init__(**kwargs)
        self.num_inputs = num_inputs
        self.num_mfs = num_mfs
        self.num_rules = num_mfs ** num_inputs

    # call function:
    def call(self, firing_strengths):
        # get batch size:
        batch_size = tf.shape(firing_strengths)[0]

        # get total firing strength:
        total_strength = tf.reduce_sum(firing_strengths, axis = 1, keepdims = True)
        # print(f'total strength: {total_strength}')
        
        # normalize the firing strengths:
        normalized_strengths = firing_strengths / (total_strength + 1e-10)

        return normalized_strengths


SyntaxError: expected argument value expression (3223514620.py, line 42)

# **Test by Creating a Model:**

This section tests the designed layers by creating a model, adding the custom layers, and testing their operation. 

In [539]:
# define the following to be used in model generation:
num_inputs = 3
num_mfs = 3
max_values = np.array([10, 25, 50])
mf_names = ['Low', 'Medium', 'High']
params = np.array([
    [  # Parameters for input 1
        [0, 0, 6],
        [5/6, 5, 55/6],
        [4, 10, 10]
    ],
    [  # Parameters for input 2
        [0, 0 , 15],
        [25/12, 12.5, 275/12],
        [10, 25, 25]
    ],
    [  # Parameters for input 3
        [0, 0, 30],
        [25/6, 25, 275/6],
        [15, 50, 50]
    ]
])

# generate a model:
membership_layer = MembershipFunctionLayer(num_inputs = num_inputs, num_mfs = num_mfs, params = params)
firing_layer = FiringStrengthLayer(num_inputs = num_inputs, num_mfs = num_mfs)
normalize_layer = NormalizationLayer(num_inputs = num_inputs, num_mfs = num_mfs)

# , [8, 23, 48]
inputs = tf.constant([[2, 9, 21]], dtype=tf.float32)

Custom parameters have been set.


Test the membership layer:

In [540]:
fuzzified = membership_layer(inputs)
membership_layer(inputs)

<tf.Tensor: shape=(1, 3, 3), dtype=float32, numpy=
array([[[0.6666667 , 0.28000003, 0.        ],
        [0.4       , 0.66400003, 0.        ],
        [0.3       , 0.808     , 0.17142858]]], dtype=float32)>

Test the firing strength layer:

In [541]:
strength = firing_layer(fuzzified)
firing_layer(fuzzified)

<tf.Tensor: shape=(1, 27), dtype=float32, numpy=
array([[0.08000001, 0.21546668, 0.04571429, 0.13280001, 0.35767472,
        0.07588572, 0.        , 0.        , 0.        , 0.03360001,
        0.09049601, 0.0192    , 0.05577601, 0.15022339, 0.031872  ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]], dtype=float32)>

Test the normalization layer:

In [542]:
normalized = normalize_layer(strength)
normalize_layer(strength)

<tf.Tensor: shape=(1, 27), dtype=float32, numpy=
array([[0.06207765, 0.16719578, 0.03547294, 0.10304889, 0.27754503,
        0.05888508, 0.        , 0.        , 0.        , 0.02607261,
        0.07022224, 0.01489864, 0.04328054, 0.11656892, 0.02473173,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]], dtype=float32)>